In [106]:
import os
from spacy.en import English
import numpy as np

In [107]:
# config
pan15train = "/data/pan15-authorship-verification-training-dataset-english-2015-04-19/"
pan15test = "/data/pan15-authorship-verification-test-dataset2-english-2015-04-19/"
pan14train = "/data/pan14-author-verification-training-corpus-english-essays-2014-04-22/"
pan14test = "/data/pan14-author-verification-test-corpus2-english-essays-2014-04-22/"
char_embeddings_file = "glove.840B.300d-char.txt"  # full path to the character embeddings file

In [108]:
def read_file(filepath):
    with open(filepath) as f:
        s = f.read()
    return s

def load_pan_data(directory, prefix="E"):
    """Load known and unknown texts in the PAN data format"""
    # FIXME: assumes one known file per author, which is fine for English datasets only
    authors = sorted([x for x in os.listdir(directory) if x.startswith(prefix)])
    known_texts = []
    unknown_texts = []
    for author in authors:
        kf = os.path.join(directory, author, "known01.txt")
        uf = os.path.join(directory, author, "unknown.txt")
        known_texts.append(read_file(kf))
        unknown_texts.append(read_file(uf))
        
    truthfile = os.path.join(directory, "truth.txt")
    with open(truthfile) as f:
        lines = f.read().strip().split("\n")
    y = [1 if line.split()[1] == "Y" else 0 for line in lines]
    y = np.array(y)
    return known_texts, unknown_texts, y

def create_pairs(knownX, unknownX):
    print(len(knownX), len(unknownX))
    """Creates pairs of known and unknown texts"""
    pairs = []
    for i in range(len(knownX)):
        pairs += [[knownX[i], unknownX[i]]]
    pairs = np.array(pairs)
    print(pairs.shape)
    return pairs

def vectorize(text, nlp):
    """Convert text (string) to embeddings (numpy array)
    nlp should be an initialised Spacy pipeline with loaded embeddings"""
    return nlp(text, entity=False, tag=False, parse=False).vector

In [110]:
from __future__ import absolute_import
from __future__ import print_function
np.random.seed(1337)  # for reproducibility
import random

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Lambda, Embedding, LSTM, Dropout, Masking, Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras import backend as K

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    model = Sequential()
    model.add(Embedding(85, 300, weights=[embedding_matrix], input_length=5000, trainable=False))
    model.add(Masking())
    # model.add(Conv1D(64, 5, padding='valid', activation='relu', strides=1))
    model.add(LSTM(128))
    # model.add(Dense(256, activation='relu'))
    # model.add(BatchNormalization())
    # model.add(Dense(128, activation='relu'))
    # model.add(BatchNormalization())
    return model

def compute_accuracy(predictions, labels):
    return np.mean(np.equal(predictions.ravel() < 0.5, labels))

def train(tr_pairs, tr_y, epochs=10):
    input_dim = tr_pairs.shape[-1]

    # network definition
    base_network = create_base_network(input_dim)

    input_a = Input(shape=(input_dim,))
    input_b = Input(shape=(input_dim,))

    # because we re-use the same instance `base_network`,
    # the weights of the network
    # will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    model = Model(inputs=[input_a, input_b], outputs=distance)
    
    rms = RMSprop()
    model.compile(loss=contrastive_loss, optimizer=rms)
    model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
              validation_split=0.0,
              batch_size=10,
              epochs=epochs)
    return model
       
def evaluate(model, tr_pairs, tr_y, te_pairs, te_y):
    # compute final accuracy on training and test sets
    pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
    tr_acc = compute_accuracy(pred, tr_y)
    pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
    te_acc = compute_accuracy(pred, te_y)
    print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
    print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))
    
def combine_vectors(v1s, v2s):
    return [np.hstack([v1s[i], v2s[i]]) for i in range(len(v1s))]

In [111]:
import string
chars = " " + string.ascii_letters + string.punctuation  # sorted to keep indices consistent
charset = set(chars)  # for lookup
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [112]:
known_train, unknown_train, y_train = load_pan_data(pan15train)
max([len(x) for x in known_train + unknown_train])

3490

In [113]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 5000
MAX_SEQUENCE_LENGTH = 1000

known_train, unknown_train, y_train = load_pan_data(pan15train)
known_test, unknown_test, y_test = load_pan_data(pan15test)

'''
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(known_train + unknown_train + known_test + unknown_test)

known_seqs_tr = pad_sequences(tokenizer.texts_to_sequences(known_train), MAX_SEQUENCE_LENGTH)
unknown_seqs_tr = pad_sequences(tokenizer.texts_to_sequences(unknown_train), MAX_SEQUENCE_LENGTH)
known_seqs_te = pad_sequences(tokenizer.texts_to_sequences(known_test), MAX_SEQUENCE_LENGTH)
unknown_seqs_te = pad_sequences(tokenizer.texts_to_sequences(unknown_test), MAX_SEQUENCE_LENGTH)

print(known_seqs_tr[0])'''

def texts_to_sequences(texts, target_length=5000):
    sequences = []
    for text in texts:
        tokens = []
        for char in text:
            if char in char_indices:
                tokens.append(char_indices[char])
                if len(tokens) >= target_length:
                    break
        while len(tokens) < target_length:
            tokens.insert(0, 0)
        sequences.append(tokens)
    return sequences

def get_embeddings(embeddings_path):
    embedding_vectors = {}
    with open(embeddings_path, 'r') as f:
        for line in f:
            line_split = line.strip().split(" ")
            vec = np.array(line_split[1:], dtype=float)
            char = line_split[0]
            embedding_vectors[char] = vec

    embedding_matrix = np.zeros((len(chars), 300))
    for char, i in char_indices.items():
        embedding_vector = embedding_vectors.get(char)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

known_seqs_tr = texts_to_sequences(known_train)
unknown_seqs_tr = texts_to_sequences(unknown_train)
known_seqs_te = texts_to_sequences(known_test)
unknown_seqs_te = texts_to_sequences(unknown_test)

In [114]:
embeddings_path = "glove.840B.300d-char.txt"
embedding_matrix = get_embeddings(embeddings_path)

In [115]:
tr_pairs = create_pairs(known_seqs_tr, unknown_seqs_tr)
te_pairs = create_pairs(known_seqs_te, unknown_seqs_te)

100 100
(100, 2, 5000)
500 500
(500, 2, 5000)


In [68]:
def run_experiment(known_train, unknown_train, known_test, unknown_test, y_train, y_test):
    tr_pairs = create_pairs(known_train, unknown_train)
    te_pairs = create_pairs(known_test, unknown_test)
    
    model = train(tr_pairs, y_train)
    evaluate(model, tr_pairs, y_train, te_pairs, y_test)

In [116]:
model = train(tr_pairs, y_train, 3)

Epoch 1/3
100/100 [==============================] - 169s - loss: 0.3121     
Epoch 2/3
100/100 [==============================] - 168s - loss: 0.2605     

In [79]:
model.fit([te_pairs[:, 0], te_pairs[:, 1]], y_test,
              validation_split=0.0,
              batch_size=10,
              epochs=1)

Epoch 1/1
500/500 [==============================] - 829s - loss: 0.1799     

In [101]:
evaluate(model, tr_pairs, y_train, te_pairs, y_test)

ValueError: operands could not be broadcast together with shapes (12800,) (100,) 

In [105]:
y_test.shape

(500,)

In [80]:
# known_train, unknown_train, y_train = load_pan_data(pan14train)
# known_test, unknown_test, y_test = load_pan_data(pan14test)
run_experiment(known_seqs_te, unknown_seqs_te, known_seqs_tr, unknown_seqs_tr, y_test, y_train)

500 500
(500, 2, 1000)
100 100
(100, 2, 1000)
Train on 450 samples, validate on 50 samples
Epoch 1/15
450/450 [==============================] - 294s - loss: 0.2708 - val_loss: 0.3293

KeyboardInterrupt: 